###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Ministerio de Defensa Nacional de Colombia

In [1]:
import pandas as pd

###Archivo yi5j-5fe9.csv

Contiene información sobre actos de terrorismo, definido como el que provoque o mantenga en estado de zozobra o terror a la población o a un sector de ella, mediante actos que pongan en peligro la vida, la integridad física o la libertad de las personas o las edificaciones o medios de comunicación, transporte, procesamiento o conducción de fluidos o fuerzas motrices, valiéndose de medios capaces de causar estragos (Ley 599 de 2000, artículo 343). Se mide en casos.

In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/yi5j-5fe9.csv?$limit=500000", dtype={'cod_muni': str})
df.head()

,fecha_hecho,cod_depto,departamento,cod_muni,municipio,zona,cantidad
0,2024-07-19T00:00:00.000,76,VALLE DEL CAUCA,76364,JAMUNDI,RURAL,1
1,2024-07-17T00:00:00.000,50,META,50330,MESETAS,URBANA,1
2,2024-07-14T00:00:00.000,5,ANTIOQUIA,05134,CAMPAMENTO,URBANA,1
3,2024-07-04T00:00:00.000,54,NORTE DE SANTANDER,54820,TOLEDO,URBANA,1
4,2024-07-03T00:00:00.000,5,ANTIOQUIA,05647,SAN ANDRES DE CUERQUIA,RURAL,1


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5694 entries, 0 to 5693
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   fecha_hecho   5694 non-null   object
 1   cod_depto     5694 non-null   int64 
 2   departamento  5694 non-null   object
 3   cod_muni      5694 non-null   object
 4   municipio     5694 non-null   object
 5   zona          5694 non-null   object
 6   cantidad      5694 non-null   int64 
dtypes: int64(2), object(5)
memory usage: 311.5+ KB


### Eliminación de columnas irrelevantes para el proyecto

In [4]:
relevant_cols = ['cod_muni', 'departamento', 'municipio', 'zona', 'fecha_hecho', 'cantidad']
terrorism_MinDefensa = df[relevant_cols]
terrorism_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5694 entries, 0 to 5693
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cod_muni      5694 non-null   object
 1   departamento  5694 non-null   object
 2   municipio     5694 non-null   object
 3   zona          5694 non-null   object
 4   fecha_hecho   5694 non-null   object
 5   cantidad      5694 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 267.0+ KB


### Convertir columna fecha_hecho a tipo date

In [5]:
# Hacer una copia explícita del DataFrame
terrorism_MinDefensa_ = terrorism_MinDefensa.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
terrorism_MinDefensa_['fecha_hecho'] = pd.to_datetime(terrorism_MinDefensa_['fecha_hecho'], errors='coerce')

# Extraer solo el año
terrorism_MinDefensa_['fecha_hecho'] = terrorism_MinDefensa_['fecha_hecho'].dt.year

In [6]:
terrorism_MinDefensa_['fecha_hecho'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010], dtype=int32)

### Verificar valores nulos

In [8]:
terrorism_MinDefensa_.isnull().sum()

,0
cod_muni,0
departamento,0
municipio,0
zona,0
fecha_hecho,0
cantidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [9]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['departamento', 'municipio', 'zona']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(terrorism_MinDefensa_[column].unique())
    print()


Categorías en la columna 'departamento':
['VALLE DEL CAUCA' 'META' 'ANTIOQUIA' 'NORTE DE SANTANDER' 'NARIÑO'
 'CAUCA' 'HUILA' 'CASANARE' 'CHOCO' 'GUAVIARE' 'ATLANTICO' 'SANTANDER'
 'CESAR' 'ARAUCA' 'BOLIVAR' 'CAQUETA' 'SUCRE' 'MAGDALENA' 'CORDOBA'
 'PUTUMAYO' 'LA GUAJIRA' 'BOGOTA D.C.' 'QUINDIO' 'TOLIMA' 'CUNDINAMARCA'
 'RISARALDA' 'CALDAS' 'VICHADA' 'BOYACA' 'AMAZONAS']

Categorías en la columna 'municipio':
['JAMUNDI' 'MESETAS' 'CAMPAMENTO' 'TOLEDO' 'SAN ANDRES DE CUERQUIA'
 'TARAZA' 'EL BAGRE' 'TAMINANGO' 'URRAO' 'GRANADA' 'ACACIAS' 'PUERTO RICO'
 'SANTANDER DE QUILICHAO' 'YARUMAL' 'CALOTO' 'NEIVA' 'POPAYAN' 'POLICARPA'
 'ITUANGO' 'JAMBALO' 'BURITICA' 'HATO COROZAL' 'CACERES' 'FRONTINO'
 'YOPAL' 'BOJAYA' 'VALDIVIA' 'SIPI' 'MORALES' 'EL CASTILLO' 'SEGOVIA'
 'REMEDIOS' 'ANGOSTURA' 'SACAMA' 'SAN JOSE DEL GUAVIARE' 'BARRANQUILLA'
 'NUNCHIA' 'DONMATIAS' 'CALI' 'SANTA ROSA DE OSOS' 'VILLAVICENCIO'
 'EL RETORNO' 'ALGECIRAS' 'GARZON' 'AMALFI' 'ANORI' 'CAMPOALEGRE'
 'CASTILLA LA NUEVA' 'BARR

- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [10]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [11]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    terrorism_MinDefensa_[col] = terrorism_MinDefensa_[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

In [12]:
# Reemplazar valores en la columna 'zona'
terrorism_MinDefensa_.loc[:, 'zona'] = terrorism_MinDefensa_['zona'].replace({'URBANA': 1, 'RURAL': 0})

### Verificar que los valores en "cod_muni" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [13]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "cod_muni" en el df terrorism_MinDefensa_

In [14]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
terrorism_MinDefensa_['cod_muni'] = terrorism_MinDefensa_['cod_muni'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = terrorism_MinDefensa_['cod_muni'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [15]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [16]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,cod_muni,count,percentage
0,5,5694,1.0


In [17]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = terrorism_MinDefensa_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_muni']])
    print()

Muestra de registros con longitud 5:
  cod_muni
0    76364
1    50330
2    05134
3    54820
4    05647



Nota: el 100%  de los codigos que identifican a los municipios en el df terrorism_MinDefensa corresponden a string de 5 caracteres, igual a los almacenados en la base de datos del proyecto

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

Se aplica la función para racionalizar columnas categoricas (Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños) al df 'dept_mpiios_codes'


In [18]:
# Aplicar función a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

In [19]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [20]:
# Comparar listas de departamento
compare_lists(terrorism_MinDefensa_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en terrorism_MinDefensa_", "Departamentos dept_mpios_codes")

Departamentos en terrorism_MinDefensa_ que no están en Departamentos dept_mpios_codes:
set()


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [21]:
# Comparar listas de códigos
compare_lists(terrorism_MinDefensa_['cod_muni'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en terrorism_MinDefensa_", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en terrorism_MinDefensa_ que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [22]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_terrorism_MinDefensa = terrorism_MinDefensa_.drop(columns=columns_to_drop)

In [23]:
# Adicionar columna para trazabilidad de la fuente
final_terrorism_MinDefensa['source_id'] = 25

In [24]:
final_terrorism_MinDefensa.columns

Index(['cod_muni', 'zona', 'fecha_hecho', 'cantidad', 'source_id'], dtype='object')

In [25]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'cod_muni': 'dane_code',
    'fecha_hecho': 'year_of_incident',
    'zona': 'area',
    'cantidad': 'amount',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_terrorism_MinDefensa.rename(columns=translation_map, inplace=True)

In [26]:
#Estructura final del dataset a integrar a la base de datos
final_terrorism_MinDefensa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5694 entries, 0 to 5693
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   dane_code         5694 non-null   object
 1   area              5694 non-null   object
 2   year_of_incident  5694 non-null   int32 
 3   amount            5694 non-null   int64 
 4   source_id         5694 non-null   int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 200.3+ KB


## Salvar en archivo csv en el drive

In [27]:
# Guardar en archivos CSV en el drive
final_terrorism_MinDefensa.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/terrorism_MinDefensa.csv', index=False)